In [1]:
include("TidalLoveNumbers.jl")
using .TidalLoveNumbers

# Get precision of Love number module (e.g., Float64, Double64, etc)
prec = TidalLoveNumbers.prec    # Type of real numbers
precc = TidalLoveNumbers.precc  # TYpe of complex numbers

ComplexF64 (alias for Complex{Float64})

# Total Heating Rate

In [2]:
# Rotational and orbital parameters
ω = 5.31e-5     # Orbital frequency (2π / orbital period)
ecc = 0.05      # Orbtial eccentricity

# Internal structure:
# First element is the innermost layer, last element is the outermost layer

# Create a toy, homogeneous Enceladus-ish model
ρ = prec[3300, 3300, 3300, 3300]  # Bulk density of each layer (kg m^-3)
r = prec[0,                             # Radii of each boundary (km)
         1,                             # Radius of CMB. If no core, set this to a 1km.
         200,                           
         230, 
         251.2] .* 1e3                  # Outer radius of solid surface 
μ = prec[60, 60, 60, 60] .* 1e9         # Elastic shear modulus of each layer (Pa)
κ = prec[100, 100, 100, 100] .* 1e16    # Bulk modulus of each layer (set to a large number for incompressible material) (GPa)
η = prec[1e18, 1e18, 1e18, 1e18]        # Shear viscosity of each layer (Pa s)  

μc = 1im * μ*ω ./ (1im*ω .+ μ./η)       # Complex shear modulus for a Maxwell material. Change this for different rheologies.

R = r[end]

r = expand_layers(r, nr=20)
g = get_g(r, ρ);

In [3]:
tidal_solution = calculate_y(r, ρ, g, μc, κ)       # Get "y-functions"

k2 = tidal_solution[5, end, end] - 1               # Get k2 Tidal Love Number 

Edot = get_bulk_heating(tidal_solution, ω, R, ecc) # Get total power output in watts
println("Total dissipation = ", Edot/1e9, " GW")

# Do sanity check/check for deviation from a homogeneous material
k2_analytical = 1.5 / (1 + 19/2 * μc[end] / (ρ[end]*g[end,end]*R))
println("Numerical error = ", abs(k2 - k2_analytical)/abs(k2_analytical)*100, "%")

Total dissipation = 0.09483883964185143 GW
Numerical error = 0.5691389274202364%
